In [1]:
#Boruta install
!pip install boruta

    100% |████████████████████████████████| 61kB 2.4MB/s ta 0:00:011
  Running setup.py bdist_wheel for boruta ... done
  Stored in directory: /home/yando/.cache/pip/wheels/5c/5a/72/13e8ea10ba10e22e9ca7f76f8b451c9f98fa190d428c8857dd
Successfully built boruta
You are using pip version 9.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
from boruta_py import BorutaPy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.ensemble import RandomForestClassifier as RandomForest
from sklearn.metrics import confusion_matrix

train_df = pd.read_csv('Titanic-Kaggle-Competition-master/train.csv')
test_df = pd.read_csv('Titanic-Kaggle-Competition-master/test.csv')
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [2]:
train_df['Age'] = train_df['Age'].fillna(train_df['Age'].median())
encoded_Pclass = pd.get_dummies(train_df['Pclass'], prefix= train_df['Pclass'].name, prefix_sep="_")
encoded_Sex = pd.get_dummies(train_df['Sex'], prefix= train_df['Sex'].name, prefix_sep="_")
encoded_Embarked = pd.get_dummies(train_df['Embarked'], prefix= train_df['Embarked'].name, prefix_sep="_")

train_df['Cabin'] = train_df['Cabin'].map(lambda x:str(x)[0])
encoded_Cabin = pd.get_dummies(train_df['Cabin'], prefix= train_df['Cabin'].name, prefix_sep="_")

train_df = train_df.drop(['Pclass','Sex','Embarked','Cabin'], axis=1)
train_df = train_df.join([encoded_Pclass,encoded_Sex,encoded_Embarked,encoded_Cabin])

train_df = train_df.drop(['Name','PassengerId', 'Ticket'], axis=1)

train_df

,Survived,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,...,Embarked_S,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_n
0,0,22.0,1,0,7.2500,0,0,1,0,1,...,1,0,0,0,0,0,0,0,0,1
1,1,38.0,1,0,71.2833,1,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
2,1,26.0,0,0,7.9250,0,0,1,1,0,...,1,0,0,0,0,0,0,0,0,1
3,1,35.0,1,0,53.1000,1,0,0,1,0,...,1,0,0,1,0,0,0,0,0,0
4,0,35.0,0,0,8.0500,0,0,1,0,1,...,1,0,0,0,0,0,0,0,0,1
5,0,28.0,0,0,8.4583,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,1
6,0,54.0,0,0,51.8625,1,0,0,0,1,...,1,0,0,0,0,1,0,0,0,0
7,0,2.0,3,1,21.0750,0,0,1,0,1,...,1,0,0,0,0,0,0,0,0,1
8,1,27.0,0,2,11.1333,0,0,1,1,0,...,1,0,0,0,0,0,0,0,0,1
9,1,14.0,1,0,30.0708,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [3]:
X = train_df.drop(['Survived'],axis=1).values
y = train_df['Survived'].values

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [4]:
from sklearn.metrics import (roc_curve, auc, accuracy_score)

rf = RandomForest(random_state=0)
rf = rf.fit(X_train, y_train)
pred = rf.predict(X_test)
fpr, tpr, thresholds = roc_curve(y_test, pred, pos_label=1)
auc(fpr, tpr)
accuracy_score(pred, y_test)

0.7982062780269058

In [5]:
print(rf.classes_)
print(confusion_matrix(y_test,rf.predict(X_test),labels=rf.classes_))

[0 1]
[[123  18]
 [ 27  55]]


In [6]:
#boruta perc=10
feat_selector = BorutaPy(rf,n_estimators='auto', perc=10, two_step=False, verbose=2, random_state=0)
feat_selector.fit(X_train, y_train)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	9 / 100
Confirmed: 	18
Tentative: 	2
Rejected: 	1
Iteration: 	10 / 100
Confirmed: 	18
Tentative: 	2
Rejected: 	1
Iteration: 	11 / 100
Confirmed: 	18
Tentative: 	2
Rejected: 	1
Iteration: 	12 / 100
Confirmed: 	18
Tentative: 	2
Rejected: 	1
Iteration: 	13 / 100
Confirmed: 	18
Tentative: 	2
Rejected: 	1
Iteration: 	14 / 100
Confirmed: 	18
Tentative: 	2
Rejected: 	1
Iteration: 	15 / 100
Confirmed: 	18
Tentative: 	2
Rejected: 	1
Iteration: 	16 / 100
Confirmed: 	18
Tentative: 	2
Rejected: 	1
I

BorutaPy(alpha=0.05,
     estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=61, n_jobs=1,
            oob_score=False,
            random_state=<mtrand.RandomState object at 0x7fc38f7acc18>,
            verbose=0, warm_start=False),
     max_iter=100, n_estimators='auto', perc=10,
     random_state=<mtrand.RandomState object at 0x7fc38f7acc18>,
     two_step=False, verbose=2)

In [7]:
mask = feat_selector.support_
X_train_selected = X_train[:, mask]
X_test_selected = X_test[:, mask]
mask

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
       False, False,  True])

In [8]:
rf = RandomForest(random_state=0)
rf = rf.fit(X_train_selected, y_train)
pred = rf.predict(X_test_selected)
fpr, tpr, thresholds = roc_curve(y_test, pred, pos_label=1)
auc(fpr, tpr)
accuracy_score(pred, y_test)

0.7937219730941704

In [9]:
print(rf.classes_)
print(confusion_matrix(y_test,rf.predict(X_test_selected),labels=rf.classes_))

[0 1]
[[126  15]
 [ 31  51]]


In [10]:
#boruta perc=20
feat_selector = BorutaPy(rf,n_estimators='auto', perc=20, two_step=False, verbose=2, random_state=0)
feat_selector.fit(X_train, y_train)

mask = feat_selector.support_
X_train_selected = X_train[:, mask]
X_test_selected = X_test[:, mask]

rf = RandomForest(random_state=0)
rf = rf.fit(X_train_selected, y_train)
pred = rf.predict(X_test_selected)
fpr, tpr, thresholds = roc_curve(y_test, pred, pos_label=1)
auc(fpr, tpr)
accuracy_score(pred, y_test)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	9 / 100
Confirmed: 	14
Tentative: 	4
Rejected: 	3
Iteration: 	10 / 100
Confirmed: 	14
Tentative: 	4
Rejected: 	3
Iteration: 	11 / 100
Confirmed: 	14
Tentative: 	4
Rejected: 	3
Iteration: 	12 / 100
Confirmed: 	14
Tentative: 	4
Rejected: 	3
Iteration: 	13 / 100
Confirmed: 	14
Tentative: 	4
Rejected: 	3
Iteration: 	14 / 100
Confirmed: 	14
Tentative: 	4
Rejected: 	3
Iteration: 	15 / 100
Confirmed: 	14
Tentative: 	4
Rejected: 	3
Iteration: 	16 / 100
Confirmed: 	14
Tentative: 	4
Rejected: 	3
I

0.8026905829596412

In [11]:
#boruta perc=30
feat_selector = BorutaPy(rf,n_estimators='auto', perc=30, two_step=False, verbose=2, random_state=0)
feat_selector.fit(X_train, y_train)

mask = feat_selector.support_
X_train_selected = X_train[:, mask]
X_test_selected = X_test[:, mask]

rf = RandomForest(random_state=0)
rf = rf.fit(X_train_selected, y_train)
pred = rf.predict(X_test_selected)
fpr, tpr, thresholds = roc_curve(y_test, pred, pos_label=1)
auc(fpr, tpr)
accuracy_score(pred, y_test)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	9 / 100
Confirmed: 	13
Tentative: 	4
Rejected: 	4
Iteration: 	10 / 100
Confirmed: 	13
Tentative: 	4
Rejected: 	4
Iteration: 	11 / 100
Confirmed: 	13
Tentative: 	4
Rejected: 	4
Iteration: 	12 / 100
Confirmed: 	13
Tentative: 	4
Rejected: 	4
Iteration: 	13 / 100
Confirmed: 	13
Tentative: 	4
Rejected: 	4
Iteration: 	14 / 100
Confirmed: 	13
Tentative: 	4
Rejected: 	4
Iteration: 	15 / 100
Confirmed: 	13
Tentative: 	4
Rejected: 	4
Iteration: 	16 / 100
Confirmed: 	13
Tentative: 	2
Rejected: 	6
I

0.8071748878923767

In [12]:
#boruta perc=40
feat_selector = BorutaPy(rf,n_estimators='auto', perc=40, two_step=False, verbose=2, random_state=0)
feat_selector.fit(X_train, y_train)

mask = feat_selector.support_
X_train_selected = X_train[:, mask]
X_test_selected = X_test[:, mask]

rf = RandomForest(random_state=0)
rf = rf.fit(X_train_selected, y_train)
pred = rf.predict(X_test_selected)
fpr, tpr, thresholds = roc_curve(y_test, pred, pos_label=1)
auc(fpr, tpr)
accuracy_score(pred, y_test)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	9 / 100
Confirmed: 	12
Tentative: 	3
Rejected: 	6
Iteration: 	10 / 100
Confirmed: 	12
Tentative: 	3
Rejected: 	6
Iteration: 	11 / 100
Confirmed: 	12
Tentative: 	3
Rejected: 	6
Iteration: 	12 / 100
Confirmed: 	12
Tentative: 	3
Rejected: 	6
Iteration: 	13 / 100
Confirmed: 	12
Tentative: 	1
Rejected: 	8
Iteration: 	14 / 100
Confirmed: 	12
Tentative: 	1
Rejected: 	8
Iteration: 	15 / 100
Confirmed: 	12
Tentative: 	1
Rejected: 	8
Iteration: 	16 / 100
Confirmed: 	12
Tentative: 	1
Rejected: 	8
I

0.7892376681614349

In [13]:
#boruta perc=50
feat_selector = BorutaPy(rf,n_estimators='auto', perc=50, two_step=False, verbose=2, random_state=0)
feat_selector.fit(X_train, y_train)

mask = feat_selector.support_
X_train_selected = X_train[:, mask]
X_test_selected = X_test[:, mask]

rf = RandomForest(random_state=0)
rf = rf.fit(X_train_selected, y_train)
pred = rf.predict(X_test_selected)
fpr, tpr, thresholds = roc_curve(y_test, pred, pos_label=1)
auc(fpr, tpr)
accuracy_score(pred, y_test)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	9 / 100
Confirmed: 	10
Tentative: 	2
Rejected: 	9
Iteration: 	10 / 100
Confirmed: 	10
Tentative: 	2
Rejected: 	9
Iteration: 	11 / 100
Confirmed: 	10
Tentative: 	2
Rejected: 	9
Iteration: 	12 / 100
Confirmed: 	10
Tentative: 	2
Rejected: 	9
Iteration: 	13 / 100
Confirmed: 	10
Tentative: 	2
Rejected: 	9
Iteration: 	14 / 100
Confirmed: 	10
Tentative: 	2
Rejected: 	9
Iteration: 	15 / 100
Confirmed: 	10
Tentative: 	2
Rejected: 	9
Iteration: 	16 / 100
Confirmed: 	10
Tentative: 	2
Rejected: 	9
I

0.7713004484304933

In [14]:
#boruta perc=60
feat_selector = BorutaPy(rf,n_estimators='auto', perc=60, two_step=False, verbose=2, random_state=0)
feat_selector.fit(X_train, y_train)

mask = feat_selector.support_
X_train_selected = X_train[:, mask]
X_test_selected = X_test[:, mask]

rf = RandomForest(random_state=0)
rf = rf.fit(X_train_selected, y_train)
pred = rf.predict(X_test_selected)
fpr, tpr, thresholds = roc_curve(y_test, pred, pos_label=1)
auc(fpr, tpr)
accuracy_score(pred, y_test)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	9 / 100
Confirmed: 	9
Tentative: 	3
Rejected: 	9
Iteration: 	10 / 100
Confirmed: 	9
Tentative: 	3
Rejected: 	9
Iteration: 	11 / 100
Confirmed: 	9
Tentative: 	3
Rejected: 	9
Iteration: 	12 / 100
Confirmed: 	9
Tentative: 	3
Rejected: 	9
Iteration: 	13 / 100
Confirmed: 	9
Tentative: 	3
Rejected: 	9
Iteration: 	14 / 100
Confirmed: 	9
Tentative: 	3
Rejected: 	9
Iteration: 	15 / 100
Confirmed: 	9
Tentative: 	3
Rejected: 	9
Iteration: 	16 / 100
Confirmed: 	9
Tentative: 	3
Rejected: 	9
Iteration

0.8071748878923767

In [15]:
#boruta perc=70
feat_selector = BorutaPy(rf,n_estimators='auto', perc=70, two_step=False, verbose=2, random_state=0)
feat_selector.fit(X_train, y_train)

mask = feat_selector.support_
X_train_selected = X_train[:, mask]
X_test_selected = X_test[:, mask]

rf = RandomForest(random_state=0)
rf = rf.fit(X_train_selected, y_train)
pred = rf.predict(X_test_selected)
fpr, tpr, thresholds = roc_curve(y_test, pred, pos_label=1)
auc(fpr, tpr)
accuracy_score(pred, y_test)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	9 / 100
Confirmed: 	9
Tentative: 	3
Rejected: 	9
Iteration: 	10 / 100
Confirmed: 	9
Tentative: 	3
Rejected: 	9
Iteration: 	11 / 100
Confirmed: 	9
Tentative: 	3
Rejected: 	9
Iteration: 	12 / 100
Confirmed: 	9
Tentative: 	3
Rejected: 	9
Iteration: 	13 / 100
Confirmed: 	9
Tentative: 	3
Rejected: 	9
Iteration: 	14 / 100
Confirmed: 	9
Tentative: 	3
Rejected: 	9
Iteration: 	15 / 100
Confirmed: 	9
Tentative: 	3
Rejected: 	9
Iteration: 	16 / 100
Confirmed: 	9
Tentative: 	2
Rejected: 	10
Iteratio

0.8071748878923767

In [16]:
#boruta perc=80
feat_selector = BorutaPy(rf,n_estimators='auto', perc=80, two_step=False, verbose=2, random_state=0)
feat_selector.fit(X_train, y_train)

mask = feat_selector.support_
X_train_selected = X_train[:, mask]
X_test_selected = X_test[:, mask]

rf = RandomForest(random_state=0)
rf = rf.fit(X_train_selected, y_train)
pred = rf.predict(X_test_selected)
fpr, tpr, thresholds = roc_curve(y_test, pred, pos_label=1)
auc(fpr, tpr)
accuracy_score(pred, y_test)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	9 / 100
Confirmed: 	8
Tentative: 	3
Rejected: 	10
Iteration: 	10 / 100
Confirmed: 	8
Tentative: 	3
Rejected: 	10
Iteration: 	11 / 100
Confirmed: 	8
Tentative: 	3
Rejected: 	10
Iteration: 	12 / 100
Confirmed: 	8
Tentative: 	3
Rejected: 	10
Iteration: 	13 / 100
Confirmed: 	9
Tentative: 	1
Rejected: 	11
Iteration: 	14 / 100
Confirmed: 	9
Tentative: 	1
Rejected: 	11
Iteration: 	15 / 100
Confirmed: 	9
Tentative: 	1
Rejected: 	11
Iteration: 	16 / 100
Confirmed: 	9
Tentative: 	0
Rejected: 	12



0.8071748878923767

In [17]:
#boruta perc=90
feat_selector = BorutaPy(rf,n_estimators='auto', perc=90, two_step=False, verbose=2, random_state=0)
feat_selector.fit(X_train, y_train)

mask = feat_selector.support_
X_train_selected = X_train[:, mask]
X_test_selected = X_test[:, mask]

rf = RandomForest(random_state=0)
rf = rf.fit(X_train_selected, y_train)
pred = rf.predict(X_test_selected)
fpr, tpr, thresholds = roc_curve(y_test, pred, pos_label=1)
auc(fpr, tpr)
accuracy_score(pred, y_test)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	9 / 100
Confirmed: 	6
Tentative: 	3
Rejected: 	12
Iteration: 	10 / 100
Confirmed: 	6
Tentative: 	3
Rejected: 	12
Iteration: 	11 / 100
Confirmed: 	6
Tentative: 	3
Rejected: 	12
Iteration: 	12 / 100
Confirmed: 	6
Tentative: 	3
Rejected: 	12
Iteration: 	13 / 100
Confirmed: 	6
Tentative: 	3
Rejected: 	12
Iteration: 	14 / 100
Confirmed: 	6
Tentative: 	3
Rejected: 	12
Iteration: 	15 / 100
Confirmed: 	6
Tentative: 	3
Rejected: 	12
Iteration: 	16 / 100
Confirmed: 	6
Tentative: 	3
Rejected: 	12
I

0.7757847533632287

In [18]:
#boruta perc=100
feat_selector = BorutaPy(rf,n_estimators='auto', perc=100, two_step=False, verbose=2, random_state=0)
feat_selector.fit(X_train, y_train)

mask = feat_selector.support_
X_train_selected = X_train[:, mask]
X_test_selected = X_test[:, mask]

rf = RandomForest(random_state=0)
rf = rf.fit(X_train_selected, y_train)
pred = rf.predict(X_test_selected)
fpr, tpr, thresholds = roc_curve(y_test, pred, pos_label=1)
auc(fpr, tpr)
accuracy_score(pred, y_test)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	21
Rejected: 	0
Iteration: 	9 / 100
Confirmed: 	4
Tentative: 	0
Rejected: 	17


BorutaPy finished running.

Iteration: 	10 / 100
Confirmed: 	4
Tentative: 	0
Rejected: 	17


0.7757847533632287